In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [3]:
df = describe_dataset("./train.csv")
df.loc[df["label"] == "shallow_squat", "label"] = 0
df.loc[df["label"] == "deep_squat", "label"] = 1
df.tail(3)

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 1652 
Number of columns: 37

Labels: 
shallow_squat    918
deep_squat       734
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_knee_z,right_knee_v,left_ankle_x,left_ankle_y,left_ankle_z,left_ankle_v,right_ankle_x,right_ankle_y,right_ankle_z,right_ankle_v
1649,1,0.475554,0.303452,0.157256,0.999904,0.343780,0.388515,0.007881,0.999794,0.511688,...,0.193412,0.727817,0.334455,0.713003,0.692403,0.470211,0.489185,0.735192,0.330466,0.678011
1650,1,0.474504,0.303404,0.140882,0.999929,0.342789,0.388602,-0.015362,0.999830,0.513757,...,0.203753,0.710959,0.332642,0.719050,0.683486,0.469902,0.490640,0.733807,0.385629,0.624616
1651,1,0.475810,0.304429,0.131367,0.999933,0.345322,0.392606,-0.016306,0.999853,0.513180,...,0.168201,0.712924,0.333597,0.715677,0.664751,0.485422,0.490398,0.734532,0.310663,0.642040


In [4]:
# Extract features and class
X = df.drop("label", axis=1)
y = df["label"].astype("int")

In [5]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(3)

1205    1
1270    1
622     0
Name: label, dtype: int32

In [7]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []

for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=[0, 1, 2])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=[0, 1, 2])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=[0, 1, 2])
    cm = confusion_matrix(y_test, model_results, labels=[0, 1, 2])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))

In [8]:
# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)

pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,RF,"[0.918, 0.926, 0.0]",0.921450,"[0.947, 0.887, 0.0]","[0.933, 0.906, 0.0]","[[180, 10, 0], [16, 125, 0], [0, 0, 0]]"
1,SVC,"[0.9, 0.865, 0.0]",0.885196,"[0.9, 0.865, 0.0]","[0.9, 0.865, 0.0]","[[171, 19, 0], [19, 122, 0], [0, 0, 0]]"
2,KNN,"[0.899, 0.859, 0.0]",0.882175,"[0.895, 0.865, 0.0]","[0.897, 0.862, 0.0]","[[170, 20, 0], [19, 122, 0], [0, 0, 0]]"
3,LR,"[0.895, 0.858, 0.0]",0.879154,"[0.895, 0.858, 0.0]","[0.895, 0.858, 0.0]","[[170, 20, 0], [20, 121, 0], [0, 0, 0]]"
4,SGDC,"[0.862, 0.844, 0.0]",0.854985,"[0.889, 0.809, 0.0]","[0.876, 0.826, 0.0]","[[169, 21, 0], [27, 114, 0], [0, 0, 0]]"
5,DTC,"[0.873, 0.787, 0.0]",0.833837,"[0.832, 0.837, 0.0]","[0.852, 0.811, 0.0]","[[158, 32, 0], [23, 118, 0], [0, 0, 0]]"
6,NB,"[0.924, 0.655, 0.0]",0.761329,"[0.637, 0.929, 0.0]","[0.754, 0.768, 0.0]","[[121, 69, 0], [10, 131, 0], [0, 0, 0]]"


In [9]:
test_df = describe_dataset("./test.csv")
test_df = test_df.sample(frac=1).reset_index(drop=True)

test_df.loc[test_df["label"] == "shallow_squat", "label"] = 0
test_df.loc[test_df["label"] == "deep_squat", "label"] = 1

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"].astype("int")

test_x = pd.DataFrame(sc.transform(test_x))

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 543 
Number of columns: 37

Labels: 
shallow_squat    304
deep_squat       239
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [10]:
testset_final_results = []

for name, model in models.items():
    # Evaluate model
    model_results = model.predict(test_x)

    p_score = precision_score(test_y, model_results, average=None, labels=[0, 1, 2])
    a_score = accuracy_score(test_y, model_results)
    r_score = recall_score(test_y, model_results, average=None, labels=[0, 1, 2])
    f1_score_result = f1_score(test_y, model_results, average=None, labels=[0, 1, 2])
    cm = confusion_matrix(test_y, model_results, labels=[0, 1, 2])
    testset_final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm ))


testset_final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(testset_final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,SVC,"[0.876, 0.806, 0.0]",0.843462,"[0.839, 0.849, 0.0]","[0.857, 0.827, 0.0]","[[255, 49, 0], [36, 203, 0], [0, 0, 0]]"
1,SGDC,"[0.853, 0.822, 0.0]",0.839779,"[0.862, 0.812, 0.0]","[0.858, 0.817, 0.0]","[[262, 42, 0], [45, 194, 0], [0, 0, 0]]"
2,LR,"[0.855, 0.812, 0.0]",0.836096,"[0.852, 0.816, 0.0]","[0.853, 0.814, 0.0]","[[259, 45, 0], [44, 195, 0], [0, 0, 0]]"
3,RF,"[0.812, 0.786, 0.0]",0.801105,"[0.839, 0.753, 0.0]","[0.825, 0.769, 0.0]","[[255, 49, 0], [59, 180, 0], [0, 0, 0]]"
4,KNN,"[0.815, 0.759, 0.0]",0.790055,"[0.809, 0.766, 0.0]","[0.812, 0.763, 0.0]","[[246, 58, 0], [56, 183, 0], [0, 0, 0]]"
5,DTC,"[0.794, 0.743, 0.0]",0.771639,"[0.799, 0.736, 0.0]","[0.797, 0.739, 0.0]","[[243, 61, 0], [63, 176, 0], [0, 0, 0]]"
6,NB,"[0.812, 0.595, 0.0]",0.677716,"[0.553, 0.837, 0.0]","[0.658, 0.696, 0.0]","[[168, 136, 0], [39, 200, 0], [0, 0, 0]]"


In [11]:
with open("./model/all_sklearn.pkl", "wb") as f:
    pickle.dump(models, f)

In [12]:
with open("./model/SGDC_model.pkl", "wb") as f:
    pickle.dump(models["SGDC"], f)

In [13]:
with open("./model/LR_model.pkl", "wb") as f:
    pickle.dump(models["LR"], f)

In [14]:
with open("./model/SVC_model.pkl", "wb") as f:
    pickle.dump(models["SVC"], f)

In [15]:
# Dump input scaler
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)